# Counter reports

Пример отчета с расчетом статистик по хитам и уникам по счетчику

In [ ]:
import sys
sys.path.append("../../mediascope-api/")


In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.counter import tasks

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

## Обязательный фильтр по типу клиента - areaTypeFilter
- audience - площадки
- advertisingCampaign - рекламные кампании
- advertisingNetwork - рекламные сети

Задается списком:

```
area_type_filter = ["audience", "advertisingCampaign", "advertisingNetwork"]
```

## Расчет хитов и уников по VK за июль 2022

In [ ]:
counter_task = tasks.CounterTask()

# Задаем название ресурса для отображения в DataFrame
project_name = 'vk-month-2022-07'


# Задаем период
date_filter = [('2022-07-01', '2022-07-31')]

# Задаем фильтр по типу клиента (партнера), в данном случае перечисляем все:
# Фильтр обязательный
area_type_filter = ["audience", "advertisingCampaign", "advertisingNetwork"]

# Задаем список клиентов (партнеров) 
partner_filter = ["vk_com"]

# Фильтр по tmsec, задается списком, в нашем случае не требуется
tmsec_filter = None

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Фильтр по типам устройств
device_type_filter=None

# Указываем список срезов, чтобы сформировать структуру расчета
slices=["partnerName", "tmsec"]

# Указываем список статистик для расчета
statistics = ["hitsVisits", "uniqsVisits"] 
#statistics = ["hitsVisits"] 

# Задаем процент данных на которых будет выполнен расчет
# Для периода больше одного дня sampling не может быть равен 100%
# sampling по умолчанию = 42 :)
sampling=25

# Формируем задание для API Cross Web в формате JSON
task_json = counter_task.build_task(task_name=project_name, 
                                    date_filter=date_filter,
                                    area_type_filter=area_type_filter,
                                    partner_filter=partner_filter, 
                                    tmsec_filter=tmsec_filter,
                                    geo_filter=geo_filter,
                                    device_type_filter=device_type_filter,
                                    slices=slices,
                                    statistics=statistics,
                                    sampling=sampling)

df_result = counter_task.result2table(counter_task.send_task(task_json), project_name)
df_result
#task_json

## Отчет по хитам уникам за стуки по России

In [ ]:
counter_task = tasks.CounterTask()

# Задаем название ресурса для отображения в DataFrame
project_name = 'vk-2022-08-01-russia'


# Задаем период
date_filter = [('2022-08-01', '2022-08-01')]

# Задаем фильтр по типу клиента (партнера), в данном случае перечисляем все:
# Фильтр обязательный
area_type_filter = ["audience", "advertisingCampaign", "advertisingNetwork"]

# Задаем список клиентов (партнеров) 
partner_filter = ["vk_com"]

# Фильтр по tmsec, задается списком, в нашем случае не требуется
tmsec_filter = None

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = "countryName = 'РОССИЯ'"


# Фильтр по типам устройств
device_type_filter=None


# Указываем список срезов, чтобы сформировать структуру расчета
slices=["researchDate", "partnerName", "tmsec"]

# Указываем список статистик для расчета
statistics = ["hitsVisits", "uniqsVisits"] 
#statistics = ["hitsVisits"] 

# Задаем процент данных на которых будет выполнен расчет
# Для периода больше одного дня sampling не может быть равен 100%
# sampling по умолчанию = 42 :)
sampling=100

# Формируем задание для API Cross Web в формате JSON
task_json = counter_task.build_task(task_name=project_name, 
                                    date_filter=date_filter,
                                    area_type_filter=area_type_filter,
                                    partner_filter=partner_filter, 
                                    tmsec_filter=tmsec_filter,
                                    geo_filter=geo_filter,
                                    device_type_filter=device_type_filter,
                                    slices=slices,
                                    statistics=statistics,
                                    sampling=sampling)

df_result = counter_task.result2table(counter_task.send_task(task_json), project_name)
df_result

## Пример отчет за стуки по России с разбивкой по городам

In [ ]:
counter_task = tasks.CounterTask()

# Задаем название ресурса для отображения в DataFrame
project_name = 'vk-2022-08-01-russia-by-cites'


# Задаем период
date_filter = [('2022-08-01', '2022-08-01')]

# Задаем фильтр по типу клиента (партнера), в данном случае перечисляем все:
# Фильтр обязательный
area_type_filter = ["audience", "advertisingCampaign", "advertisingNetwork"]

# Задаем список клиентов (партнеров) 
partner_filter = ["vk_com"]

# Фильтр по tmsec, задается списком, в нашем случае не требуется
tmsec_filter = None

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = "countryName = 'РОССИЯ'"
#geo_filter = "countryName = 'РОССИЯ' and cityName='Москва'"

# Фильтр по типам устройств
device_type_filter=None


# Указываем список срезов, чтобы сформировать структуру расчета
slices=["researchDate", "partnerName", "tmsec", "countryName", "cityName"]

# Указываем список статистик для расчета
statistics = ["hitsVisits", "uniqsVisits"] 
#statistics = ["hitsVisits"] 

# Задаем процент данных на которых будет выполнен расчет
# Для периода больше одного дня sampling не может быть равен 100%
# sampling по умолчанию = 42 :)
sampling=100

# Формируем задание для API Cross Web в формате JSON
task_json = counter_task.build_task(task_name=project_name, 
                                    date_filter=date_filter,
                                    area_type_filter=area_type_filter,
                                    partner_filter=partner_filter, 
                                    tmsec_filter=tmsec_filter,
                                    geo_filter=geo_filter,
                                    device_type_filter=device_type_filter,
                                    slices=slices,
                                    statistics=statistics,
                                    sampling=sampling)

df_result = counter_task.result2table(counter_task.send_task(task_json), project_name)
df_result

## Пример отчета с разбивкой по типам устройств

In [ ]:
counter_task = tasks.CounterTask()

# Задаем название ресурса для отображения в DataFrame
project_name = 'vk-2022-08-01-by-device-type'


# Задаем период
date_filter = [('2022-08-01', '2022-08-01')]

# Задаем фильтр по типу клиента (партнера), в данном случае перечисляем все:
# Фильтр обязательный
area_type_filter = ["audience", "advertisingCampaign", "advertisingNetwork"]

# Задаем список клиентов (партнеров) 
partner_filter = ["vk_com"]

# Фильтр по tmsec, задается списком, в нашем случае не требуется
tmsec_filter = ["vk_videostart", "vk_videoend"]

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = "countryName = 'РОССИЯ'"
#geo_filter = "countryName = 'РОССИЯ' and cityName='Москва'"

# Фильтр по типам устройств
device_type_filter=None


# Указываем список срезов, чтобы сформировать структуру расчета
slices=["researchDate", "partnerName", "tmsec", "deviceTypeName"]

# Указываем список статистик для расчета
statistics = ["hitsVisits", "uniqsVisits"] 
#statistics = ["hitsVisits"] 

# Задаем процент данных на которых будет выполнен расчет
# Для периода больше одного дня sampling не может быть равен 100%
# sampling по умолчанию = 42 :)
sampling=100

# Формируем задание для API Cross Web в формате JSON
task_json = counter_task.build_task(task_name=project_name, 
                                    date_filter=date_filter,
                                    area_type_filter=area_type_filter,
                                    partner_filter=partner_filter, 
                                    tmsec_filter=tmsec_filter,
                                    geo_filter=geo_filter,
                                    device_type_filter=device_type_filter,
                                    slices=slices,
                                    statistics=statistics,
                                    sampling=sampling)

df_result = counter_task.result2table(counter_task.send_task(task_json), project_name)
df_result.sort_values(by=['tmsec', 'deviceTypeName'])

## Пример отчет за стуки по мобильным устройствам с разбивкой по городам

In [ ]:
counter_task = tasks.CounterTask()

# Задаем название ресурса для отображения в DataFrame
project_name = 'vk-2022-08-01-by-device-type'


# Задаем период
date_filter = [('2022-08-01', '2022-08-01')]

# Задаем фильтр по типу клиента (партнера), в данном случае перечисляем все:
# Фильтр обязательный
area_type_filter = ["audience", "advertisingCampaign", "advertisingNetwork"]

# Задаем список клиентов (партнеров) 
partner_filter = ["vk_com"]

# Фильтр по tmsec, задается списком, в нашем случае не требуется
tmsec_filter = ["vk_videostart", "vk_videoend"]

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Фильтр по типам устройств
device_type_filter="deviceTypeName in ('ANDROID', 'IOS', 'SMARTPHONE', 'SMALLSCREEN', 'TABLET', 'MOBILE')"


# Указываем список срезов, чтобы сформировать структуру расчета
slices=["researchDate", "partnerName", "tmsec", "cityName"]

# Указываем список статистик для расчета
statistics = ["hitsVisits", "uniqsVisits"] 
#statistics = ["hitsVisits"] 

# Задаем процент данных на которых будет выполнен расчет
# Для периода больше одного дня sampling не может быть равен 100%
# sampling по умолчанию = 42 :)
sampling=100

# Формируем задание для API Cross Web в формате JSON
task_json = counter_task.build_task(task_name=project_name, 
                                    date_filter=date_filter,
                                    area_type_filter=area_type_filter,
                                    partner_filter=partner_filter, 
                                    tmsec_filter=tmsec_filter,
                                    geo_filter=geo_filter,
                                    device_type_filter=device_type_filter,
                                    slices=slices,
                                    statistics=statistics,
                                    sampling=sampling)

df_result = counter_task.result2table(counter_task.send_task(task_json), project_name)
df_result

## Пример отчет за стуки по "другим" устройствам

In [ ]:
counter_task = tasks.CounterTask()

# Задаем название ресурса для отображения в DataFrame
project_name = 'vk-2022-09-10-by-device-type-other'


# Задаем период
date_filter = [('2022-09-10', '2022-09-10')]

# Задаем фильтр по типу клиента (партнера), в данном случае перечисляем все:
# Фильтр обязательный
area_type_filter = ["audience", "advertisingCampaign", "advertisingNetwork"]

# Задаем список клиентов (партнеров) 
partner_filter = ["vk_com"]

# Фильтр по tmsec, задается списком, в нашем случае не требуется
tmsec_filter = None

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None
#geo_filter = "countryName = 'РОССИЯ' and cityName='Москва'"

# Фильтр по типам устройств
device_type_filter="deviceTypeName nin ( 'DESKTOP', 'MOBILE', 'TABLET', 'SMARTPHONE', 'IOS', 'ANDROID',  'SMALLSCREEN', 'WINDOWS_MOBILE')"

# Указываем список срезов, чтобы сформировать структуру расчета
slices=["researchDate", "partnerName", "tmsec", "cityName"]
slices=["deviceTypeName"]

# Указываем список статистик для расчета
statistics = ["hitsVisits", "uniqsVisits"] 
statistics = ["hitsVisits"] 

# Задаем процент данных на которых будет выполнен расчет
# Для периода больше одного дня sampling не может быть равен 100%
# sampling по умолчанию = 42 :)
sampling=50

# Формируем задание для API Cross Web в формате JSON
task_json = counter_task.build_task(task_name=project_name, 
                                    date_filter=date_filter,
                                    area_type_filter=area_type_filter,
                                    partner_filter=partner_filter, 
                                    tmsec_filter=tmsec_filter,
                                    geo_filter=geo_filter,
                                    device_type_filter=device_type_filter,
                                    slices=slices,
                                    statistics=statistics,
                                    sampling=sampling)

df_result = counter_task.result2table(counter_task.send_task(task_json), project_name)
df_result